In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk # SkLearn ML library
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
import seaborn as sns
import requests
from datetime import datetime

# Input data files are available in the "austin-bike" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "austin-bike"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# load data in 

trips = pd.read_csv('austin-bike/austin_bikeshare_trips.csv')
stations = pd.read_csv('austin-bike/austin_bikeshare_stations.csv')
weather = pd.read_csv('austin-bike/austin_weather.csv')


austin_bikeshare_stations.csv
austin_bikeshare_trips.csv
austin_weather.csv



In [4]:
Rain = []
Thunderstorm = []
Fog = []
Snow = []
for row in weather['Events']:
    if 'Rain' in row:
        Rain.append(1)
    else:
        Rain.append(0)
        
    if 'Thunderstorm' in row:
        Thunderstorm.append(1)
    else:
        Thunderstorm.append(0)
        
    if 'Fog' in row:
        Fog.append(1)
    else:
        Fog.append(0)
        
    if 'Snow' in row:
        Snow.append(1)
    else:
        Snow.append(0)
weather['PrecipitationSumInches'] = np.where(weather['PrecipitationSumInches'] == 'T', 0.001, weather['PrecipitationSumInches'])
weather['Rain'] = Rain
weather['Thunderstorm'] = Thunderstorm
weather['Fog'] = Fog
weather['Snow'] = Snow

In [5]:
weather.head()

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,...,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events,Rain,Thunderstorm,Fog,Snow
0,2013-12-21,74,60,45,67,49,43,93,75,57,...,2,20,4,31,0.46,"Rain , Thunderstorm",1,1,0,0
1,2013-12-22,56,48,39,43,36,28,93,68,43,...,5,16,6,25,0,,0,0,0,0
2,2013-12-23,58,45,32,31,27,23,76,52,27,...,10,8,3,12,0,,0,0,0,0
3,2013-12-24,61,46,31,36,28,21,89,56,22,...,7,12,4,20,0,,0,0,0,0
4,2013-12-25,58,50,41,44,40,36,86,71,56,...,7,10,2,16,0.001,,0,0,0,0


In [14]:
# test to see if all values in a column are numeric.
pd.to_numeric(weather.WindHighMPH)


0       20
1       16
2        8
3       12
4       10
5       12
6        7
7       10
8       17
9       13
10       8
11      13
12      21
13      14
14      16
15      21
16      17
17      13
18      12
19      12
20      13
21      12
22      15
23      17
24      16
25      14
26      14
27      14
28      12
29       9
        ..
1289    14
1290    13
1291    13
1292    13
1293    12
1294    15
1295     9
1296    12
1297    12
1298    14
1299    13
1300    10
1301    15
1302    23
1303    12
1304    16
1305    14
1306    18
1307    14
1308    14
1309    13
1310    13
1311    13
1312    15
1313    15
1314    12
1315    14
1316    12
1317    13
1318    12
Name: WindHighMPH, Length: 1319, dtype: int64

In [54]:
weather.HumidityAvgPercent[weather.HumidityAvgPercent == '-'] = 0
weather.WindHighMPH[weather.WindHighMPH == '-'] = 0
weather.PrecipitationSumInches[weather.PrecipitationSumInches == 'T'] = 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
trips.head()

,bikeid,checkout_time,duration_minutes,end_station_id,end_station_name,month,start_station_id,start_station_name,start_time,subscriber_type,trip_id,year
0,8.0,19:12:00,41,2565.0,Trinity & 6th Street,3.0,2536.0,Waller & 6th St.,2015-03-19 19:12:00,Walk Up,9900082882,2015.0
1,141.0,2:06:04,6,2570.0,South Congress & Academy,10.0,2494.0,2nd & Congress,2016-10-30 02:06:04,Local365,12617682,2016.0
2,578.0,16:28:27,13,2498.0,Convention Center / 4th St. @ MetroRail,3.0,2538.0,Bullock Museum @ Congress & MLK,2016-03-11 16:28:27,Local365,9075366,2016.0
3,555.0,15:12:00,80,2712.0,Toomey Rd @ South Lamar,11.0,2497.0,Capitol Station / Congress & 11th,2014-11-23 15:12:00,24-Hour Kiosk (Austin B-cycle),9900319298,2014.0
4,86.0,15:39:13,25,3377.0,MoPac Pedestrian Bridge @ Veterans Drive,4.0,2707.0,Rainey St @ Cummings,2017-04-16 15:39:13,Walk Up,14468597,2017.0


In [23]:
pd.to_numeric(trips.start_station_id)
trips.isnull().sum()

bikeid                  723
checkout_time             0
duration_minutes          0
end_station_id        19842
end_station_name          0
month                 30752
start_station_id      19041
start_station_name        0
start_time                0
subscriber_type        2077
trip_id                   0
year                  30752
dtype: int64

In [33]:
import dateutil.parser
t = pd.to_datetime(trips.start_time)
trips['month'] = t.map(lambda x: x.month)
trips['year'] = t.map(lambda x: x.year)


In [36]:
trips.isnull().sum()

bikeid                  723
checkout_time             0
duration_minutes          0
end_station_id        19842
end_station_name          0
month                     0
start_station_id      19041
start_station_name        0
start_time                0
subscriber_type        2077
trip_id                   0
year                      0
dtype: int64

In [127]:
dailyStats = pd.DataFrame()
dailyStats['date'] = weather['Date'].values.tolist()
dailyStats['TempAvgF'] = weather['TempAvgF'].values.tolist()
dailyStats['Humidity'] = weather['HumidityAvgPercent'].values.tolist()
dailyStats['WindMPH'] = weather['WindHighMPH'].values.tolist()
dailyStats['Precipitation'] = weather['PrecipitationSumInches'].values.tolist()
dailyStats['Rain'] = Rain
dailyStats['Fog'] = Fog
dailyStats['Thunderstorm'] = Thunderstorm
dailyStats['Snow'] = Snow


In [128]:
dailyStats.tail()

,date,TempAvgF,Humidity,WindMPH,Precipitation,Rain,Fog,Thunderstorm,Snow
1314,2017-07-27,89,54,12,0,0,0,0,0
1315,2017-07-28,91,54,14,0,0,0,0,0
1316,2017-07-29,92,51,12,0,0,0,0,0
1317,2017-07-30,93,48,13,0,0,0,0,0
1318,2017-07-31,88,43,12,0,0,0,0,0


In [115]:
dailyStats['Month'] = t.map(lambda x: x.month)
dailyStats['Year'] = t.map(lambda x: x.year)
dailyStats['weekend'] = t.map(lambda x: 1 if x.dayofweek > 4 else 0)
del dailyStats['date']
dailyStats.head()

,TempAvgF,Humidity,WindMPH,Precipitation,Rain,Fog,Thunderstorm,Snow,Month,Year,weekend
0,60,75,20,0.46,1,0,1,0,3,2015,0
1,48,68,16,0,0,0,0,0,10,2016,1
2,45,52,8,0,0,0,0,0,3,2016,0
3,46,56,12,0,0,0,0,0,11,2014,1
4,50,71,10,0,0,0,0,0,4,2017,1


In [118]:
X_train = dailyStats[dailyStats.index<1107]
X_test = dailyStats[dailyStats.index>=1107]
len(X_train)

1107

## daily trip aggregates


In [129]:
# add a date column that contains 
import dateutil.parser
trips['date'] = trips['start_time'].map(lambda x: dateutil.parser.parse(x).date())
trips['date'] = trips['date'].map(lambda x: x.strftime('%Y-%m-%d'))

idx = pd.date_range('2013-12-21', '2017-07-31')

tripCount = trips.date.value_counts()
tripCount.index = pd.DatetimeIndex(tripCount.index)
tripCount = tripCount.reindex(idx, fill_value=0)

tripCount.sort_index(inplace=True)
tripCount.tail()


2017-07-27    370
2017-07-28    402
2017-07-29    425
2017-07-30    365
2017-07-31    344
Freq: D, Name: date, dtype: int64

In [132]:
tripCount.reset_index(drop=True,inplace=True)

In [131]:
print(len(tripCount))
print(len(dailyStats))

1319
1319


In [133]:
y_train = tripCount[tripCount.index<1107]
y_test = tripCount[tripCount.index>=1107]

In [134]:
print(len(y_test))
print(len(X_test))

212
212


## start some machine learning!

In [144]:
# import our machine learning libraries
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn import model_selection, preprocessing

In [98]:
# define function for measuring error
# this function will be how we measure error
# it is the root mean squared logarithmic error
def rmsle(y, y_):
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

In [135]:
clf = LinearRegression(normalize= True)
clf.fit(X_train, y_train)
print("The accuracy of linear regression is: ", clf.score(X_test, y_test), "\nThe RMSLE is: ", rmsle(y_test, clf.predict(X_test)))

The accuracy of linear regression is:  -0.059325701874802794 
The RMSLE is:  0.4865263482974606


In [142]:
clf = Ridge(alpha=0.1)
clf.fit(X_train, y_train)
print("The accuracy of linear regression is: ", clf.score(X_test, y_test), "\nThe RMSLE is: ", rmsle(y_test, clf.predict(X_test)))

The accuracy of linear regression is:  -0.0593965261177094 
The RMSLE is:  0.48654094076424653


In [143]:
clf = Lasso(alpha=0.1)
clf.fit(X_train, y_train)
print("The accuracy of linear regression is: ", clf.score(X_test, y_test), "\nThe RMSLE is: ", rmsle(y_test, clf.predict(X_test)))

The accuracy of linear regression is:  -0.059576455608631555 
The RMSLE is:  0.48648592495913273


In [136]:
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)
print("The accuracy of Decision Tree Regressor is: ", clf.score(X_test, y_test), "\nThe RMSLE is: ", rmsle(y_test, clf.predict(X_test)))

The accuracy of Decision Tree Regressor is:  -1.8527043781591226 
The RMSLE is:  1.9737036677801423


In [137]:
clf = RandomForestRegressor(n_estimators= 200)
clf.fit(X_train, y_train)
print('The accuracy of Random Forest Regressor is: ', clf.score(X_test, y_test), "\nThe RMSLE is: ", rmsle(y_test, clf.predict(X_test)))

The accuracy of Random Forest Regressor is:  -0.07881339262594445 
The RMSLE is:  0.5079731708543583
